In [2]:
from urllib.request import urlopen
import json

response = urlopen('https://statsapi.web.nhl.com/api/v1/game/2018021255/feed/live')

d = json.load(response)
from pandas.io.json import json_normalize
import pandas as pd 
df = json_normalize(d['liveData']['boxscore']['teams']['home']['players'])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [178]:
def stats_list(pid):
    response = urlopen('https://statsapi.web.nhl.com/api/v1/people/' +str(pid) + '/stats?stats=gameLog&season=20192020')
    d = json.load(response)
    
    stat = []
    for dic in d['stats'][0]['splits']:
        for key in dic['stat']:
            stat.append(key)
        break
    return stat


In [188]:
def gather_teams(): 
    response = urlopen('https://statsapi.web.nhl.com/api/v1/teams')
    d = json.load(response)

    team_dict = {'team_name':[], 'team_id':[]}
    for team in d['teams']:
        team_dict['team_name'].append(team['name'])
        team_dict['team_id'].append(team['id'])
    x = pd.DataFrame(team_dict)
    team_dict2 = {'team_name':[], 'team_id':[], 'fullName':[], 'player_id':[], 'position':[]}
    for id_ in team_dict['team_id']:
        response = urlopen('https://statsapi.web.nhl.com/api/v1/teams/' + str(id_) + '/roster')
        d = json.load(response)
        for ob in d['roster']:
            team_dict2['team_name'].append(x[x.team_id == id_].team_name.values[0])
            team_dict2['team_id'].append(id_)
            team_dict2['player_id'].append(ob['person']['id'])
            team_dict2['fullName'].append(ob['person']['fullName'])
            team_dict2['position'].append(ob['position']['abbreviation'])
    return pd.DataFrame(team_dict2) 
        
team_df = gather_teams()
team_df = pd.DataFrame(player_df)
team_df

,team_name,team_id,fullName,player_id,position
0,New Jersey Devils,1,Sami Vatanen,8475222,D
1,New Jersey Devils,1,Travis Zajac,8471233,C
2,New Jersey Devils,1,P.K. Subban,8474056,D
3,New Jersey Devils,1,Wayne Simmonds,8474190,RW
4,New Jersey Devils,1,Kyle Palmieri,8475151,RW
...,...,...,...,...,...
762,Vegas Golden Knights,54,Nate Schmidt,8477220,D
763,Vegas Golden Knights,54,Shea Theodore,8477447,D
764,Vegas Golden Knights,54,William Carrier,8477478,LW
765,Vegas Golden Knights,54,Tomas Nosek,8477931,LW


In [189]:
g_stats = stats_list(8475839)
p_stats = stats_list(8477949)
g_stats, p_stats

(['timeOnIce',
  'ot',
  'shutouts',
  'saves',
  'powerPlaySaves',
  'shortHandedSaves',
  'evenSaves',
  'shortHandedShots',
  'evenShots',
  'powerPlayShots',
  'decision',
  'savePercentage',
  'games',
  'gamesStarted',
  'shotsAgainst',
  'goalsAgainst',
  'shortHandedSavePercentage',
  'evenStrengthSavePercentage'],
 ['timeOnIce',
  'assists',
  'goals',
  'pim',
  'shots',
  'games',
  'hits',
  'powerPlayGoals',
  'powerPlayPoints',
  'powerPlayTimeOnIce',
  'evenTimeOnIce',
  'penaltyMinutes',
  'faceOffPct',
  'shotPct',
  'gameWinningGoals',
  'overTimeGoals',
  'shortHandedGoals',
  'shortHandedPoints',
  'shortHandedTimeOnIce',
  'blocked',
  'plusMinus',
  'points',
  'shifts'])

In [195]:
def stat_gather(player_df, pos, stat, season = '20192020'):
    player_scores = {}
    for keys in stats:
        player_scores[keys] = []

    player_scores['player_id'] = []
    player_scores['fullName'] = []
    player_scores['game_id'] = []
    player_scores['team_name'] = []
    player_scores['team_id'] = []
    
    if pos == 'G':
        print("Gathering Goalie Stats")
        ids = player_df[player_df.position == 'G'].player_id.values
    else:
        print("Gathering Non Goalie Stats")
        ids = player_df[player_df.position != 'G'].player_id.values
    
    for pid in ids:
        response = urlopen('https://statsapi.web.nhl.com/api/v1/people/'+str(pid)+'/stats?stats=gameLog&season='+season)
        d = json.load(response)
        for dic in d['stats'][0]['splits']:
            player_scores['game_id'].extend([dic['game']['gamePk']])
            player_scores['player_id'].extend([pid])
            player_scores['fullName'].extend([player_df[player_df.player_id == pid].fullName.values[0]])
            player_scores['team_id'].extend([dic['team']['id']])
            player_scores['team_name'].extend([dic['team']['name']])
            for key in stat:
                try:
                    player_scores[key].extend([dic['stat'][key]])
                except KeyError:
                    player_scores[key].extend([None])

    return pd.DataFrame(player_scores)

    
goalie_df = stat_gather(team_df, pos='G', stats=g_stats)
player_df = stat_gather(team_df, pos='ddfsdf', stat=p_stats)

Gathering Non Goalie Stats


In [198]:
player_df = pd.DataFrame(player_df)

In [204]:
def combine_frames(team_df, player_df, goalie_df):
    # add game number to players
    play_df = player_df.merge(team_df[['fullName', 'position']], on = 'fullName')
    
    d2 = play_df.groupby(['game_id','team_name']).first().reset_index()
    d2['game_num'] = d2.groupby('team_name').cumcount() + 1
    play_df = pd.merge(d2[['game_id', 'game_num', 'team_name']], play_df, on = ['game_id', 'team_name'])
    # add game number to goalies 
    d2 = goalie_df.groupby(['game_id','team_name']).first().reset_index()
    d2['game_num'] = d2.groupby('team_name').cumcount() + 1
    goalie_df = pd.merge(d2[['game_id', 'game_num', 'team_name']], goalie_df, on = ['game_id', 'team_name'])
    
    x = pd.concat([play_df, goalie_df], ignore_index=True, axis=0)
    return x

fin_frame = combine_frames(team_df, player_df, goalie_df) 


In [210]:
fin_frame

,game_id,game_num,team_name,timeOnIce,assists,goals,pim,shots,games,hits,...,shortHandedShots,evenShots,powerPlayShots,decision,savePercentage,gamesStarted,shotsAgainst,goalsAgainst,shortHandedSavePercentage,evenStrengthSavePercentage
0,2019020001,1.0,Ottawa Senators,15:16,0.0,1.0,0.0,3.0,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019020001,1.0,Ottawa Senators,17:33,0.0,0.0,0.0,0.0,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019020001,1.0,Ottawa Senators,19:40,0.0,0.0,0.0,1.0,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019020001,1.0,Ottawa Senators,19:27,0.0,0.0,2.0,1.0,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019020001,1.0,Ottawa Senators,15:47,1.0,0.0,0.0,1.0,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33269,2019020917,NaN,San Jose Sharks,58:36,NaN,NaN,NaN,NaN,1,NaN,...,0.0,22.0,5.0,L,0.852,1.0,27.0,4.0,NaN,81.818182
33270,2019020918,NaN,Vegas Golden Knights,60:00,NaN,NaN,NaN,NaN,1,NaN,...,0.0,25.0,2.0,W,0.926,1.0,27.0,2.0,NaN,96.000000
33271,2019020918,NaN,Washington Capitals,58:29,NaN,NaN,NaN,NaN,1,NaN,...,1.0,25.0,3.0,L,0.897,1.0,29.0,3.0,100.0,88.000000
33272,2019020919,NaN,Colorado Avalanche,63:03,NaN,NaN,NaN,NaN,1,NaN,...,0.0,27.0,0.0,O,0.852,1.0,27.0,4.0,NaN,85.185185


In [206]:
test = fin_frame.copy()
games = list(test.game_num.unique())

    
def game_fill(df):
    games = df.game_num.unique().tolist()
    players = df.fullName.unique().tolist()
    fill_dict = {}
    
    safe_key = ['game_id', 'game_num', 'position', 'team_name', 'fullName', 'player_id']
    for key in list(df):
        fill_dict[key] = []
        
    for player in players:
        games_played = df[df.fullName == player].game_num.tolist()
        fill_game = list(set(games) - set(games_played))
        position = df[df.fullName==player].position.tolist()[0]
        pid = df[df.fullName==player].player_id.tolist()[0]
        team = df[df.fullName==player].team_name.tolist()[0]
        for game in fill_game:
            # print(len(fill_game))
            try:
                fill_dict['game_id'].extend([df[(df.team_name == team) & (df.game_num == game)].game_id.tolist()[0]])
            except Exception as e:
                # print(e)
                fill_dict['game_id'].extend(['game_has_not_happened'])
            fill_dict['game_num'].extend([game]) 
            fill_dict['position'].extend([position])
            fill_dict['team_name'].extend([team])
            fill_dict['fullName'].extend([player])
            fill_dict['player_id'].extend([pid])
            for key in fill_dict:
                if key not in safe_key:
                    fill_dict[key].extend([None])
            
            
            
    return pd.DataFrame(fill_dict)

append_dict = game_fill(test)
        

In [219]:
f = fin_frame.append(append_dict, ignore_index=True).fillna(0)


In [227]:
def get_data(season = '20192020', return_separate=False):
    print("downloading team data")
    team_df = gather_teams()
    # update this so we don't have to worry about retirement 
    g_stats = stats_list(8475839)
    p_stats = stats_list(8477949)
    goalie_df = stat_gather(team_df, pos='G', stats=g_stats)
    player_df = stat_gather(team_df, pos='ddfsdf', stat=p_stats, season = season )
    
    fin_frame = combine_frames(team_df, player_df, goalie_df) 
    
    print("filling in missing games")
    df = fin_frame.copy()
    append_df = game_fill(test)
    
    f = fin_frame.append(append_dict, ignore_index=True).fillna(0)
    print("calculating fantasy points")
    f['score'] = f.apply(score, axis=1)

    f = f.dropna()
    if return_separate:
        return team_df, g_stats, p_stats, goalie_df, player_df, fin_frame, f

    return f

    

In [226]:
import sys
sys.path.insert(1, '../')
import importlib

import scripts.hockey_bots as hockey
importlib.reload(hockey)

def score(row):
    if row.position == "G":
        return hockey.goalie_points(row, row.gamesStarted)
    else:
        return hockey.player_points(row)
    
f['score'] = f.apply(score, axis=1)

f = f.dropna()
f

,game_id,game_num,team_name,timeOnIce,assists,goals,pim,shots,games,hits,...,evenShots,powerPlayShots,decision,savePercentage,gamesStarted,shotsAgainst,goalsAgainst,shortHandedSavePercentage,evenStrengthSavePercentage,score
0,2019020001,1.0,Ottawa Senators,15:16,0.0,1.0,0.0,3.0,1,4.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.010000
1,2019020001,1.0,Ottawa Senators,17:33,0.0,0.0,0.0,0.0,1,2.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.480000
2,2019020001,1.0,Ottawa Senators,19:40,0.0,0.0,0.0,1.0,1,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,-1.710000
3,2019020001,1.0,Ottawa Senators,19:27,0.0,0.0,2.0,1.0,1,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,-1.690000
4,2019020001,1.0,Ottawa Senators,15:47,1.0,0.0,0.0,1.0,1,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,4.291176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33648,2019020569,38.0,Toronto Maple Leafs,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
33649,2019020583,39.0,Toronto Maple Leafs,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
33650,2019020770,50.0,Toronto Maple Leafs,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
33651,2019020777,51.0,Toronto Maple Leafs,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [223]:
f[f.fullName == "Connor McDavid"].goals.sum()

30.0